# CNAM UASB03 - CERTIFICATION ANALYSE DE DONNEES MASSIVES
## Projet d'analyse de sentiment sur les commentaires Airbnb en français

***
Notebook Python de préparation des commentaires en entrée de notre modélisation.
*  Chargement du fichier brut
*  Détection de la langue
*  Filtrage uniquement des commentaires reconnus comme français avec plus de 80% de probabilité
*  Génération d'un fichier avec uniquement l'identifiant du commentaire (permet de remonter au fichier source) et le commentaire



In [2]:
import pandas, os, json, numpy
from whatthelang import WhatTheLang
wtl = WhatTheLang()

Airbnb_comment=pandas.read_csv("Data/reviews_Avril2017.csv",encoding='utf-8',lineterminator='\n')
Airbnb_comment.comments.str.replace("\r\n", "")
Airbnb_comment.comments.str.replace("\n", "")
Airbnb_comment.comments.str.encode(encoding='utf-8', errors='strict')
Airbnb_comment['comments']=Airbnb_comment['comments'].astype(str)
Airbnb_comment.head(10)


,listing_id,_id,date,reviewer_id,reviewer_name,comments
0,1027568,47892721,2015-09-21,4346070,Joydeep,Beautiful home in every sense; aesthetics & co...
1,1027568,49618144,2015-10-05,22681962,Marc,Iris and her family greeted us at the apartmen...
2,1027568,79705618,2016-06-13,37238215,Sam,We were greeted by Iris and her family on arri...
3,1268496,9575106,2014-01-02,48218,Collin,I stayed at Aurelie's over the new year and it...
4,1268496,13111695,2014-05-19,6667381,Kane,Auréle's place is great. It as our 2nd trip to...
5,1268496,17799682,2014-08-17,17060334,Elodie,Nous n'avons pas croisé Aurélie qui était hors...
6,1268496,19091362,2014-09-06,19253510,Krystte,The reservation was canceled 3 days before arr...
7,1268496,22880015,2014-11-16,21245715,Romain,The reservation was canceled 5 days before arr...
8,1268496,24491528,2014-12-29,24540616,Michelle,Aurélie Allet s appartment is really near the ...
9,1268496,26456093,2015-02-13,6364566,Vladymyr,Staying in the apartment of Aurélie without an...


In [13]:
Airbnb_comment.shape

(846219, 8)

In [5]:
#détection de la langue avec whatthelang
Airbnb_comment['langage']=Airbnb_comment['comments'].apply(wtl.pred_prob)
Airbnb_comment['langue']=Airbnb_comment['langage'].str[0].str[0].str[0]
Airbnb_comment['lg_proba']=Airbnb_comment['langage'].str[0].str[0].str[1]
Airbnb_comment=Airbnb_comment.drop(['listing_id', 'date', 'reviewer_id','reviewer_name','langage'],axis=1)

In [ ]:
#Récupératoin des commentaire identifiés comme français à plus de 80% de probabilité

Comment = Airbnb_comment[numpy.logical_and(Airbnb_comment['lg_proba']>=0.8,Airbnb_comment['langue']=='fr') ]

Comment['comments'] = Comment['comments'].replace({r'\r\n': ' '}, regex=True).replace("\n", " ").replace({r'\s+$': ' ', r'^\s+': ' '}, regex=True).replace(r'\n',  ' ', regex=True)
Comment = Comment.drop(['langue','lg_proba'],axis=1)

In [27]:
Comment.shape

(197372, 2)

In [28]:
Comment.head()

,_id,comments
5,17799682,Nous n'avons pas croisé Aurélie qui était hors...
38,103496190,Très bon accueil et tres attentionnée La pro...
40,55808517,"Un grand studio , avec tout le confort nécessa..."
41,59437146,"Studio très confortable, bien aménagé et jolim..."
43,12388077,Très satisfait de la location de cet apparteme...


In [29]:
# nettoyage des commentaires
Comment['comments']=Comment['comments'].str.strip('\n')
Comment['comments'] = Comment['comments'].astype(str)
Comment['comments'] = Comment['comments'].apply(lambda x: x.replace('.',' ').replace('@',' ').replace('+',' ').
            replace('&',' ').replace(':',' ').replace(':)',' ').replace('\"\"',' ').replace(',',' ').
            replace('(',' ').replace(')',' ').replace(';',' ').replace('!',' ').replace('  ',' '))

In [30]:
Comment.to_csv("Data/Commentaires_identifiants_V3",encoding='utf-8',header=None,sep='£',index=False)